In [17]:
import pandas as pd
import numpy as np
import random

#split dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#AUCROC,AUCPR,precision,recall,f1-score
from sklearn.metrics import roc_curve,auc,average_precision_score,precision_score,recall_score,f1_score

#gridsearch/randomsearch
from itertools import product
from tqdm import tqdm

#visualize results
import matplotlib.pyplot as plt
import time

In [18]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import Subset,DataLoader,TensorDataset
from torchvision import datasets,transforms
import torch.nn.functional as F

In [19]:
# import warnings
# warnings.filterwarnings("ignore")

if torch.cuda.is_available():
  is_cuda=True
  print('GPU is on')
else:
  is_cuda=False
  print('GPU is off')

GPU is off


In [20]:
def set_seed(seed):
  np.random.seed(seed)
  torch.manual_seed(seed)
  random.seed(seed)

  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

In [21]:
def data_prepare(dataset,batch_size,contam_ratio,seed):
    set_seed(seed)
    filepath = 'D:/Jiang/Research_Anomaly Detection/Comparison/Data_compare/'
    data_ia = pd.read_csv(filepath + dataset + '_ia_' + str(contam_ratio) + '_' + str(seed) + '.csv')
    data_ul = pd.read_csv(filepath + dataset + '_ul_' + str(contam_ratio) + '_' + str(seed) + '.csv')
    data_val = pd.read_csv(filepath + dataset + '_val_' + str(contam_ratio) + '_' + str(seed) + '.csv')
    data_test = pd.read_csv(filepath + dataset + '_test_' + str(contam_ratio) + '_' + str(seed) + '.csv')
        
    #将0,1label转换为1,-1
    data_ia.loc[data_ia['y'] == 1,'y'] = -1
    data_ia.loc[data_ia['y'] == 0,'y'] = 1
    
    #其中注意ul中由于label未知,必须将其全部转为1(normal)
    data_ul.loc[data_ul['y'] == 1,'y'] = 1
    data_ul.loc[data_ul['y'] == 0,'y'] = 1
    
    data_val.loc[data_val['y'] == 1,'y'] = -1
    data_val.loc[data_val['y'] == 0,'y'] = 1
    
    data_test.loc[data_test['y'] == 1,'y'] = -1
    data_test.loc[data_test['y'] == 0,'y'] = 1

    #合并ia与ul文件作为training set
    data_train = pd.concat([data_ia,data_ul],axis = 0)
    #重新对index列进行命名
    data_train.rename(columns={'Unnamed: 0':'index'},inplace = True)
    #重新按照index列进行排序,由于数据已经预先处理
    data_train = data_train.sort_values('index',ascending = True)
    data_train.reset_index(drop = True,inplace = True)
    
    #training set
    X_train = np.array(data_train.drop(['index','y'],axis = 1))
    y_train = np.array(data_train['y']).astype('float64')

    #validation set
    X_val = np.array(data_val.drop(['Unnamed: 0','y'],axis = 1))
    y_val = np.array(data_val['y']).astype('float64')

    #testing set
    X_test = np.array(data_test.drop(['Unnamed: 0','y'],axis = 1))
    y_test = np.array(data_test['y']).astype('float64')

    #to tensor
    #GAN only use "normal"samples in training phase, which may be contaminated
    index_subset = np.arange(len(y_train))
    train_tensor = TensorDataset(torch.from_numpy(X_train).float(),torch.tensor(y_train))
    train_tensor = Subset(train_tensor,index_subset)
    train_loader = DataLoader(train_tensor,batch_size = batch_size,shuffle=False,drop_last=True)

    #validation set
    X_val_tensor = torch.from_numpy(X_val).float()
    #testing set
    X_test_tensor=torch.from_numpy(X_test).float()

    return train_loader,X_val_tensor,y_val,X_test_tensor,y_test

In [22]:
class generator(nn.Module):
  def __init__(self,input_size,act_fun):
    super(generator,self).__init__()

    self.encoder_1=nn.Sequential(
      nn.Linear(input_size,input_size//4),
      act_fun,
      )
    
    self.decoder_1=nn.Sequential(
      nn.Linear(input_size//4,input_size),
      )

    self.encoder_2=nn.Sequential(
      nn.Linear(input_size,input_size//4),
      act_fun,
      )

  def forward(self,input):
    z=self.encoder_1(input)
    X_hat=self.decoder_1(z)
    z_hat=self.encoder_2(X_hat)

    return z,X_hat,z_hat

In [23]:
class discriminator(nn.Module):
  def __init__(self,input_size,act_fun):
    super(discriminator,self).__init__()

    self.encoder=nn.Sequential(
        nn.Linear(input_size,input_size//4),
        act_fun,
        )

    self.classifier=nn.Sequential(
        nn.Linear(input_size//4,1),
        nn.Sigmoid()
        )
    
  def forward(self,input):
    latent_vector=self.encoder(input)
    output=self.classifier(latent_vector)

    return latent_vector,output

In [24]:
def fit(dataloader,net_generator,net_discriminator,eta,epochs,batch_size,print_loss):
  L1_criterion = nn.L1Loss(reduction='none')
  L2_criterion = nn.MSELoss(reduction='none')
  BCE_criterion = nn.BCELoss(reduction='mean')

  if is_cuda:
    L1_criterion.cuda()
    L2_criterion.cuda()
    BCE_criterion.cuda()

  loss_D_list = []
  loss_G_list = []

  for epoch in range(epochs):
    for i,data in enumerate(dataloader):
      #y_aclabel means the acquired label information (which may be contaminated)
      X,y_aclabel = data
      y_real = torch.FloatTensor(batch_size).fill_(0)#real label=0,size=batch_size
      y_fake = torch.FloatTensor(batch_size).fill_(1)#fake label=1,size=batch_size

      if is_cuda:
        X = X.cuda()
        y_aclabel = y_aclabel.cuda()
        y_real = y_real.cuda()
        y_fake = y_fake.cuda()

      X = Variable(X)
      y_aclabel = Variable(y_aclabel)
      y_real = Variable(y_real)
      y_fake = Variable(y_fake)

      #zero grad for discriminator
      net_discriminator.zero_grad()
      #training the discriminator with real sample
      _,output = net_discriminator(X)
      loss_D_real = BCE_criterion(output.view(-1),y_real)

      #training the discriminator with fake sample
      _,X_hat,_ = net_generator(X)
      _,output = net_discriminator(X_hat)

      loss_D_fake = BCE_criterion(output.view(-1),y_fake)

      #entire loss in discriminator
      loss_D = (loss_D_real+loss_D_fake)/2
      
      loss_D.backward()
      optimizer_D.step()

      #training the generator based on the result from the discriminator
      net_generator.zero_grad()

      z,X_hat,z_hat = net_generator(X)

      #latent loss
      feature_real,_ = net_discriminator(X)
      feature_fake,_ = net_discriminator(X_hat)

      loss_G_latent = torch.mean(L2_criterion(feature_fake,feature_real),1)
      
      #contexutal loss
      loss_G_contextual = torch.mean(L1_criterion(X,X_hat),1)
      #entire loss in generator

      #encoder loss
      loss_G_encoder = torch.mean(L1_criterion(z,z_hat),1)
      
      loss_G = (loss_G_latent + loss_G_contextual + loss_G_encoder)/3
      
      loss_G_normal = (loss_G[y_aclabel == 1])
      loss_G_anomaly = pow(loss_G[y_aclabel == -1],-1)
      
      if loss_G_anomaly.size(0)>0:
        loss_G = (1-eta)*(sum(loss_G_normal)/loss_G_normal.size(0))+\
                eta*(sum(loss_G_anomaly)/loss_G_anomaly.size(0))
      else:
        loss_G = sum(loss_G_normal)/loss_G_normal.size(0)
      
      loss_G.backward()
      optimizer_G.step()

      if (i%50 == 0) & print_loss:
        print('[%d/%d] [%d/%d] Loss D: %.4f / Loss G: %.4f' % (epoch+1,epochs,i,len(dataloader),loss_D,loss_G))

      loss_D_list.append(loss_D)
      loss_G_list.append(loss_G)

In [25]:
def evaluation(data_tensor,model):
  if is_cuda:
    data_tensor = data_tensor.cuda()
    
  z,_,z_hat = model(data_tensor)

  L1_criterion = nn.L1Loss(reduction='none')
  score = L1_criterion(z,z_hat)
  score = torch.sum(score,dim=1).cpu().detach().numpy()
    
  return score

Global Parameters

In [26]:
#dataset_list = ['arrhythmia','cardio','ionosphere','mnist','optdigits','pendigits','pima','shuttle','CCFD','TBP']
#dataset_list = ['cardio','mnist','optdigits','pendigits','CCFD','TBP']
dataset_list = ['TBP']
#小数据集
#dataset_list = ['arrhythmia','ionosphere','pima']

In [27]:
contam_ratio_pool = [1.0,0.98,0.5,0.0]
seed_pool = [1,2,3,4,5]

#hybrid hyper-parameter
eta = 0.5
#random search size
search_size = 10

In [28]:
epochs = 20
batch_size = 64

hyper_act_fun = [nn.Tanh(),nn.LeakyReLU()]
#hyper_act_fun = [nn.Tanh()]
hyper_lr = [1e-2,1e-3,1e-4]
hyper_mom = [0.5,0.7,0.9]

hyper_list_entire = list(product(hyper_act_fun,hyper_lr,hyper_mom))

In [29]:
def random_search(hyper_list_entire,search_size,seed):
  if search_size < len(hyper_list_entire):
    set_seed(seed)
    index = np.random.choice(np.arange(len(hyper_list_entire)),search_size,replace = False)

    hyper_list = []
    for i in index:
      hyper_list.append(hyper_list_entire[i])
  else:
    hyper_list = hyper_list_entire

  return hyper_list

In [30]:
for dataset in dataset_list:
    print(f'*****Current Dataset:{dataset}*****')
    #save results
    df_result = pd.DataFrame(data = None,index = contam_ratio_pool,columns = seed_pool)
    for contam_ratio in tqdm(contam_ratio_pool):  
      for seed in tqdm(seed_pool):
        #############################################seleting the best hyper-parameters in validation set#############################################
        metric_value_list = list()
        hyper_list = random_search(hyper_list_entire,search_size,seed)
        for i in range(len(hyper_list)):
          try:
            print(f'Finding Optimal Hyper-parameters......Current Candidates: {hyper_list[i]}')
            act_fun,lr,mom = hyper_list[i]
            #data
            train_loader,X_val_tensor,y_val,_,_ = data_prepare(dataset,batch_size,contam_ratio,seed)
            #model initialization
            set_seed(seed)
            net_generator=generator(input_size = X_val_tensor.size(1),act_fun = act_fun)
            net_discriminator=discriminator(input_size = X_val_tensor.size(1),act_fun = act_fun)

            if is_cuda:
              net_generator.cuda()
              net_discriminator.cuda()

            optimizer_G=torch.optim.SGD(net_generator.parameters(),lr = lr,momentum = mom)
            optimizer_D=torch.optim.SGD(net_discriminator.parameters(),lr = lr,momentum = mom)
            #fitting
            fit(dataloader = train_loader,net_generator = net_generator,net_discriminator = net_discriminator,
                eta = eta,epochs = epochs,batch_size = batch_size,print_loss = False)

            #evaluation
            score = evaluation(data_tensor = X_val_tensor,model = net_generator)
            metric_value = average_precision_score(y_true = y_val,y_score = score,pos_label = -1)
            metric_value_list.append(metric_value)

            print(f'The metric value corresponded to the hyper-parameters is :{metric_value:{.4}}')
            print('******************************')
            print('\n')
          except:
            #keep the right index
            metric_value_list.append(0)
            pass
          continue

        best_hyper_params=hyper_list[metric_value_list.index(max(metric_value_list))]
        print(f'The best hyper-parameters are: {best_hyper_params}')
        print('\n')
        ###################################################################testing#########################################################################
        print('Testing Phrase......')
        act_fun,lr,mom = best_hyper_params

        #data
        train_loader,_,_,X_test_tensor,y_test = data_prepare(dataset,batch_size,contam_ratio,seed)

        #model initialization, there exists randomness because of weight initialization***
        set_seed(seed)
        net_generator=generator(input_size = X_test_tensor.size(1),act_fun = act_fun)
        net_discriminator=discriminator(input_size = X_test_tensor.size(1),act_fun = act_fun)

        if is_cuda:
          net_generator.cuda()
          net_discriminator.cuda()

        optimizer_G = torch.optim.SGD(net_generator.parameters(),lr = lr,momentum = mom)
        optimizer_D = torch.optim.SGD(net_discriminator.parameters(),lr = lr,momentum = mom)
        #fitting
        fit(dataloader = train_loader,net_generator = net_generator,net_discriminator = net_discriminator,
            eta = eta,epochs = epochs,batch_size = batch_size,print_loss = False)
        #evaluation
        score = evaluation(data_tensor = X_test_tensor,model = net_generator)

        #store the result
        #AUCPR
        df_result.loc[contam_ratio,seed] = average_precision_score(y_true = y_test,y_score = score,pos_label = -1)

    #mean & sd
    df_result['mean'] = np.mean(df_result.loc[:,seed_pool],axis = 1)
    df_result['std'] = np.std(df_result.loc[:,seed_pool],axis = 1)
    df_result = round(df_result.astype('float64')*100,2)

    file_path = 'D:/Jiang/Research_Anomaly Detection/Comparison/result/' + dataset+ '_HGAN' + '.csv'
    df_result.to_csv(file_path,index = True)

*****Current Dataset:TBP*****


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.0001, 0.5)
The metric value corresponded to the hyper-parameters is :0.01704
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.001, 0.5)
The metric value corresponded to the hyper-parameters is :0.01542
******************************


Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(negative_slope=0.01), 0.001, 0.7)
The metric value corresponded to the hyper-parameters is :0.009273
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.01, 0.9)
The metric value corresponded to the hyper-parameters is :0.03672
******************************


Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(negative_slope=0.01), 0.001, 0.9)
The metric value corresponded to the hyper-parameters is :0.007937
******************************


Finding Optimal Hyper-parameters......Current Candidates: (T


 20%|████████████████▊                                                                   | 1/5 [01:25<05:41, 85.34s/it]

Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(negative_slope=0.01), 0.01, 0.5)
The metric value corresponded to the hyper-parameters is :0.01943
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.001, 0.7)
The metric value corresponded to the hyper-parameters is :0.04558
******************************


Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(negative_slope=0.01), 0.0001, 0.7)
The metric value corresponded to the hyper-parameters is :0.03384
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.01, 0.5)
The metric value corresponded to the hyper-parameters is :0.03891
******************************


Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(negative_slope=0.01), 0.01, 0.7)
The metric value corresponded to the hyper-parameters is :0.02175
******************************


Finding Optimal Hyper-parameters......Cu

...


 60%|██████████████████████████████████████████████████▍                                 | 3/5 [03:13<02:09, 64.81s/it]

Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.0001, 0.5)
The metric value corresponded to the hyper-parameters is :0.007678
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.001, 0.5)
The metric value corresponded to the hyper-parameters is :0.007889
******************************


Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(negative_slope=0.01), 0.0001, 0.7)
The metric value corresponded to the hyper-parameters is :0.008096
******************************


Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(negative_slope=0.01), 0.01, 0.9)
The metric value corresponded to the hyper-parameters is :0.1509
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.001, 0.7)
The metric value corresponded to the hyper-parameters is :0.008138
******************************


Finding Optimal Hyper-parameters......Current Candidates: 


 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [04:22<01:05, 65.94s/it]

Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.001, 0.9)
The metric value corresponded to the hyper-parameters is :0.03412
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.01, 0.7)
The metric value corresponded to the hyper-parameters is :0.1175
******************************


Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(negative_slope=0.01), 0.01, 0.7)
The metric value corresponded to the hyper-parameters is :0.1199
******************************


Finding Optimal Hyper-parameters......Current Candidates: (Tanh(), 0.01, 0.9)
The metric value corresponded to the hyper-parameters is :0.1179
******************************


Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(negative_slope=0.01), 0.01, 0.9)
The metric value corresponded to the hyper-parameters is :0.1544
******************************


Finding Optimal Hyper-parameters......Current Candidates: (LeakyReLU(n


100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [25:43<00:00, 383.14s/it]
